# Calculate cosine similarities and generate recommendations

### Imports

In [1]:
import os
import glob
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

### Load npy files and song ids for all songs in data set

In [2]:
np_arrays = np.load('all_feature_arrays_original_data_set.npy')

In [3]:
with open('all_song_ids_original_data_set.pkl', 'rb') as handle:
    song_ids = pickle.load(handle)

In [4]:
np_arrays.shape

(8715, 128)

In [5]:
len(song_ids)

8715

### Calculate cosine similarities

In [6]:
cs = cosine_similarity(np_arrays)

In [7]:
dists = pd.DataFrame(cs, columns=song_ids)
dists.index = dists.columns

In [8]:
dists.head()

,100001,10003,100035,100039,100042,100053,100068,100077,100082,100118,...,99187,99253,99259,99351,99380,99416,99470,99473,99496,99928
100001,1.000000,-0.154758,0.702616,-0.397046,0.406874,0.452844,0.099104,0.307161,0.940820,0.979642,...,0.151751,0.476339,0.442607,0.564234,0.017172,0.144805,0.769451,0.497425,0.077625,-0.027161
10003,-0.154758,1.000000,0.004686,0.883299,0.318349,-0.212148,-0.218137,-0.373612,0.004743,-0.084599,...,0.172331,-0.128291,-0.278291,0.292668,0.652501,0.656895,0.172539,-0.143816,0.287689,0.213292
100035,0.702616,0.004686,1.000000,-0.186422,0.316381,0.014930,-0.225187,0.015335,0.670139,0.637489,...,0.607677,0.170575,-0.010678,0.328670,0.127565,0.140862,0.442408,0.286417,0.122609,-0.028630
100039,-0.397046,0.883299,-0.186422,1.000000,0.248173,-0.176949,-0.032846,-0.279605,-0.313267,-0.310417,...,0.258158,-0.043845,-0.303766,0.079623,0.467876,0.640574,-0.105737,-0.270808,0.320915,0.265737
100042,0.406874,0.318349,0.316381,0.248173,1.000000,0.041336,-0.090751,-0.247024,0.512963,0.454773,...,0.384140,0.114457,-0.101812,0.706866,0.549142,0.284744,0.682829,0.376825,0.652485,0.510134


### Load track info

In [9]:
with open(r"final_data.pkl", "rb") as input_file:
    tracks = pickle.load(input_file)

I downloaded and processed data for more than 9 genres in the end, so I just need to filter out the genres I didn't end up using.

In [10]:
parent_genres = ['Breakbeat', 'Dancehall/Ragga', 'Downtempo', 'Drum And Bass', 
                'Funky House', 'Hip Hop/R&B', 'Minimal House','Rock/Indie', 'Trance']

In [11]:
tracks = tracks[tracks['parent_genre'].isin(parent_genres)]

In [13]:
tracks.sample(5)

,id,release_artist,release_cat_num,release_date,release_genre,release_label,release_title,track_name,track_url,parent_genre
17116,132282,ARISEN FLAME,ASOT 230,30 Sep 13,Uplifting Trance,A State Of Trance Holland,Gladius,Gladius (original mix) - (7:26),http://www.junodownload.com/MP3/SF2301945-02-0...,Trance
17936,28057,ABSOLUTE,ASOT 035,30 Sep 04,Uplifting Trance,A State Of Trance Holland,Day Dream,Day Dream (original mix) - (9:40),http://www.junodownload.com/MP3/SF1270951-02-0...,Trance
13218,415459,D-UNITY,TER 02,05 May 07,Minimal/Tech House,Tigereye Recordings,The Tribal Fury EP,Children Of House (Tigereye Ritual dub) - (7:57),http://www.junodownload.com/MP3/SF3229864-02-0...,Minimal House
994,74317,ALEX WICKED,SPK 268,21 Nov 16,Breakbeat,Spektra,Warlock - Wolf,Warlock - (4:50),http://www.junodownload.com/MP3/SF3243299-02-0...,Breakbeat
5520,174235,BACKSTREET CHEMIST,JC 0025,08 Aug 12,Drum And Bass,Just Cause Recordings,Notational Sedation,Notational Sedation - (5:13),http://www.junodownload.com/MP3/SF2023427-02-0...,Drum And Bass


### Make recommendations

In [14]:
selected_song_id = '132282'

In [15]:
recommendations = dists[str(selected_song_id)].sort_values(ascending=False).reset_index()

In [16]:
recommendations.head(5)

,index,132282
0,132282,1.000000
1,53831,0.965931
2,29064,0.960896
3,246948,0.957631
4,341720,0.956003


In [17]:
top_5_recs = list(recommendations.head(6)['index'])[1:]

In [18]:
top_5_recs

['53831', '29064', '246948', '341720', '249363']

In [19]:
d_list = []
for r in top_5_recs:
    df = tracks[tracks['id'] == int(r)]    
    d = df.to_dict(orient='records')
    d_list.append(d)

In [20]:
d_list

[[{'id': 53831,
   'parent_genre': 'Trance',
   'release_artist': 'AIMOON',
   'release_cat_num': 'ETR 106',
   'release_date': '23 May 11',
   'release_genre': 'Uplifting Trance',
   'release_label': 'Eternity Recordings',
   'release_title': 'Aimoonia',
   'track_name': 'Aimoonia - (8:27)',
   'track_url': 'http://www.junodownload.com/MP3/SF1750757-02-01-01.mp3'}],
 [{'id': 29064,
   'parent_genre': 'Trance',
   'release_artist': 'GAL ABUTBUL feat PHILLIPA JOY',
   'release_cat_num': 'AMSTR 020',
   'release_date': '22 Apr 13',
   'release_genre': 'Uplifting Trance',
   'release_label': 'Amsterdam Trance',
   'release_title': "Don't Mind Waiting",
   'track_name': "Don't Mind Waiting (original mix) - (5:56)",
   'track_url': 'http://www.junodownload.com/MP3/SF2185776-02-01-01.mp3'}],
 [{'id': 246948,
   'parent_genre': 'Trance',
   'release_artist': 'BLUSKAY',
   'release_cat_num': 'AEZ 075',
   'release_date': '14 Feb 14',
   'release_genre': 'Uplifting Trance',
   'release_label': 